# Project Category Identification NLP Project <a id='Data Overview'></a>

## Table of Contents <a id='2.1_Contents'></a>
* [0. Background](#0)
* [1. Data Import - Building Dataset](#1)
* [2. Target Location Selection](#2)
* [3. Data Understanding](#3)
    * [3.1 Missing Values](#3.1)
    * [3.2 Meter Type Observation](#3.2)
    * [3.3 Building Area Observation](#3.3)
    * [3.4 Building Category Observation](#3.4)
    * [3.5 Building Energy Use Intensity](#3.5)
    * [3.6 Other Building Parameters](#3.6)
* [4.0 Save Data](#4.0)

## 0. Background

I built a web scrapping tool that scraps biding and tendering information from public sectors in Ontario. The public sectors includes regions such as York Region, Peel Region, Halton Region, and cities such as City of Mississauga.

The intend of the project is to use DATA to tell a story about the Competitiveness of the public sector. The collected information are used to conduct the following analysis:

* **Public Sector Market Understanding**: an overview of the public sector markets (services, construction, and goods), how the market changes over time.

* **Competitive Information**: an overview of the performance of private companies in different public sectors. What is their wining rate comparing with their competitors. The private companies includes construction companies such as Maple Reinders Constructors Ltd. and Kenaidan Contracting Ltd., consulting companies such WSP Canada Inc., Hatch Ltd., and good supplying companies.

* **Competitiveness by Category** Different private companies in the public sectors have overlapping skill sets in the public sectors, they may also specialize a niche that company leaders and strategist may not be aware off. The tool intends to use DATA to show competitiveness of each companies in different sectors such as consulting in water & wastewater, construction in civil engineering works, etc.

The collected information includes project names and a generic category information. In order to understand "Competitiveness by Category", the original "generic category" needs to further broken down into detailed categories. There are almost 10k project collected, it is not practical to provide a detailed category to each of the project,. Further more, as more projects be collected, manual tagging the category is timing consuming.

Therefore, Natural Language Processing (NLP) is used to do the category task for current projects and any future projects.

## 0.1 Import Libraries

In [28]:
import pandas as pd
import numpy as np
import dataresource as dr

## 0.2 Import Original Dataset

In [29]:
path = "../NaturalLanguageProcessing/Data.csv"
df = pd.read_csv(path)
df.drop_duplicates(inplace=True)

## 1. Data Wrangling

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9126 entries, 0 to 48198
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   client_name         9126 non-null   object 
 1   project_name        9126 non-null   object 
 2   bid_classification  7275 non-null   object 
 3   bid_type            9101 non-null   object 
 4   bid_ID              9126 non-null   object 
 5   awarded_date        9124 non-null   object 
 6   awarded_year        9124 non-null   float64
dtypes: float64(1), object(6)
memory usage: 570.4+ KB


In [31]:
df.head()

,client_name,project_name,bid_classification,bid_type,bid_ID,awarded_date,awarded_year
0,York Region,General Contractors And Subcontractors For Sen...,Services,Pre-Qualification,PQ-20-156,05-Mar,2021.0
9,York Region,Supply And Delivery Of As-Required Spill Respo...,Services,Tender,T-20-204,16-Feb,2021.0
13,York Region,Prequalification Of Consultants For The Feasib...,Services,Pre-Qualification,PQ-20-247,25-Jan,2021.0
20,York Region,Detailed Design And Engineering Services For 1...,Services,RFP,P-20-46,11-Feb,2021.0
25,York Region,Waste Electrical And Electronic Equipment (Wee...,Services,Tender,T-20-260,18-Dec,2020.0


## 1.1 Data Exploration

### 1.1.1 Data Exploration - client_name

In [32]:
client_list = df.client_name.unique().tolist()
len(client_list)

19

In [33]:
client_list

['York Region',
 'Waterloo Region',
 'Simcoe County',
 'Peterborough County',
 'Niagara Region',
 'Halton Region',
 'Haldimand County',
 'Durham Region',
 'Dufferin County',
 'City Of Peterborough',
 'City of Orillia',
 'City of London',
 'City of Kawartha Lakes',
 'City of Hamilton',
 'City of Guelph',
 'City of Brantford',
 'City of Barrie',
 'Brant County',
 'Peel Region']

There are 19 clients included in the dataset. The client name will not be used as an input for NLP because the client name does not have any affect on the project name and project category.

### 1.1.2 Data Exploration - bid_type

In [34]:
df.bid_type.describe()

count       9101
unique        58
top       Tender
freq        1939
Name: bid_type, dtype: object

In [35]:
df.bid_type.unique().tolist()

['Pre-Qualification',
 'Tender',
 'RFP',
 'Notice of Contract Award',
 'RFI',
 'EOI',
 'RFQ',
 'Request for Tender',
 'Request for Proposal',
 'Consultant Selection',
 'Consultant Selection Stage 1',
 'Expression of Interest',
 'Request for Quote',
 'Consultant Selection Stage 2',
 'Cooperative Tender',
 'Request for Tender (Carpentry)',
 'Request For Proposal',
 'Request For PreQualification',
 'Request for Quotation',
 'RFT',
 'Request for Offer',
 'Request For Tender',
 'Request For Quotation',
 'Request for Proposal(N)',
 'RFPQ',
 nan,
 'N',
 'NRFP',
 'Request For Quotations',
 'Request for Supplier Pre-qualification',
 'ITT',
 'RFSQ',
 'RFEOI',
 'Request for Qualifications',
 'Request for Quotation (Formal)',
 'Informal Request for Quotation',
 'Request for Information',
 'Request For Supplier Qualification',
 'RFT - Public Works',
 'RFT - Construction - Stipulated Price - CCDC',
 'RFT - Construction - Unit Price',
 'RFT - Goods and Services',
 'RFP - Goods and Services',
 'RFT - 

In [36]:
df.bid_type.value_counts(dropna = False)

Tender                                          1939
RFP                                             1529
RFT                                             1258
Request for Tender                               929
RFQ                                              425
Pre-Qualification                                317
Request For Proposal                             297
Request for Proposal                             237
Request For Quotation                            195
Request For Tender                               176
RFT - Goods and Services                         157
RFPQ                                             154
RFT - Public Works                               152
Consultant Selection                             134
RFP - Goods and Services                         104
RFQ – Low Bid                                    101
ITT                                               97
RFT - Construction - Stipulated Price - CCDC      91
Request for Quotation (Formal)                

As we can see the bid types are not unified across different regions/cities, a hashmap is developed to revise values.

In [37]:
df.bid_type = df.bid_type.apply(lambda x : dr.bid_type_Standard.get(x))

In [38]:
df.bid_type.value_counts()

Request for Tender(RFT)                 5014
Request for Proposal(RFP)               2194
Request for Quotation(RFQ)              1037
Request for Pre-Qualification(RFPQ)      745
Request for Information (RFI)             61
Others                                    48
Request for Pre-Qualification(RFPQ)'       2
Name: bid_type, dtype: int64

### 1.1.3 Data Explanation - bid_classification

In [46]:
df.bid_classification.unique()

array(['Services', 'Construction', 'Goods', nan], dtype=object)

In [50]:
df.bid_classification.value_counts(dropna = False)

Services        16952
Construction    16802
NaN              9402
Goods            5051
Name: bid_classification, dtype: int64

In [55]:
df[df.bid_classification.isnull()].head()

,client_name,project_name,Category,bid_classification,bid_type,bid_ID,awarded_date,awarded_year,company_name,submitted_price,winning_status
6433,York Region,Emergency Repairs To Roads And Related Road Fa...,NaN,NaN,Tender,T-15-94,Mar 15,2016.0,"K.J. Beamish Construction Co., Limited","$2,979,666.00",0.0
6434,York Region,Emergency Repairs To Roads And Related Road Fa...,NaN,NaN,Tender,T-15-94,Mar 15,2016.0,614128 Ontario Ltd o/a Trisan Construction,"$5,532,704.00",0.0
6571,York Region,Construction Of The Midblock Collector Road An...,NaN,NaN,Tender,T-14-53,Apr 12,2016.0,Aecon Construction and Materials Limited,"$41,375,923.41",0.0
6572,York Region,Construction Of The Midblock Collector Road An...,NaN,NaN,Tender,T-14-53,Apr 12,2016.0,Brennan Paving & Construction Ltd.,"$35,811,814.16",1.0
6573,York Region,Construction Of The Midblock Collector Road An...,NaN,NaN,Tender,T-14-53,Apr 12,2016.0,Coco Paving Inc.,"$41,589,331.45",0.0


## 3.0 Data Observation

### 3.1 Obtain data
Use pandas to read the data I scrapped.

In [28]:
excel_path = '/Users/delinmu/Documents/GitHub/BidTing/result/data.xlsx'
bidding_info = pd.read_excel(excel_path, sheet_name='Ori')

In [ ]:
# a quick view of bidding_info dataframe
bidding_info.head()

### 3.2 Dataframe Observation
A couple of questions I want to know here. 
1. How many unique clients in the dataframe?
2. How many unique projects in the dataframe?
3. A table includes 3 columns: client_name, bid_id, project_name, and awarded_year for me to tell how many projects were on the project over the time span

### 3.2.1 Unique Clients
A simple value_counts() method is used to reveal the information. Alternatively, nunique() can also reveal the information.

In [39]:
# view of the data type of my dataframe
bidding_info.client_name.value_counts()

York Region               9726
Peel Region               5519
City of Guelph            5227
Waterloo Region           5067
City of Hamilton          3857
City of Brantford         3111
Halton Region             2521
City of London            2453
Durham Region             2401
Niagara Region            1777
City of Barrie            1463
Simcoe County             1295
Brant County              1004
City of Kawartha Lakes     849
City of Orillia            493
Peterborough County        439
City Of Peterborough       429
Dufferin County            330
Haldimand County           246
Name: client_name, dtype: int64

Note that value count for each client does not represent number of unique projects.

In [40]:
len(bidding_info.client_name.value_counts())

19

In total, we have 19 clients collected in the dataframe. This is good to know how many geographical segementtation  we 